In [ ]:
from utils import *

In [ ]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

In [ ]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

In [ ]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [ ]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [ ]:
from azureml.core import Datastore
from azureml.pipeline.core import PipelineData

datastore = Datastore.get(ws, datastore_name="mldata")
data_train = PipelineData('train', datastore=datastore)
data_test = PipelineData('test', datastore=datastore)

In [ ]:
from azureml.data import OutputFileDatasetConfig

data_out = OutputFileDatasetConfig(name="predictions", destination=(datastore, 'titanic/predictions'))
data_out = data_out.read_delimited_files().register_on_complete('titanic.pred')

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

step_1 = PythonScriptStep(name='Preprocessing',
                          script_name="preprocess_output.py",
                          source_directory="code",
                          arguments=[
                              "--input", data_in,
                              "--out-train", data_train,
                              "--out-test", data_test],
                          inputs=[data_in],
                          outputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)


In [ ]:
from azureml.pipeline.steps import PythonScriptStep

step_2 = PythonScriptStep(name='Training',
                          script_name="train_output.py",
                          source_directory="code",
                          arguments=[
                              "--in-train", data_train,
                              "--in-test", data_test,
                              "--output", data_out],
                          inputs=[data_train, data_test],
                          outputs=[data_out],
                          runconfig=run_conf,
                          compute_target=aml_cluster)

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step_1, step_2])

In [ ]:
pipeline.validate()

In [ ]:
from azureml.pipeline.core import PublishedPipeline

for pipeline in PublishedPipeline.list(ws):
    print("name: %s, id: %s" % (pipeline.name, pipeline.id))

pipeline = PublishedPipeline.list(ws)[0]

In [ ]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

recurrence = ScheduleRecurrence(frequency="Minute", interval=15)

schedule = Schedule.create(ws,
                           name="AzureML Scheduled Pipeline", 
                           pipeline_id=pipeline.id,
                           experiment_name="azureml-pipeline-schedule", 
                           recurrence=recurrence,
                           pipeline_parameters={})

In [ ]:
# schedule.disable()